**REPLICATION CODE: MOURIFIÉ/HENRY/MEANGO**

Start date: 10/11/2025
Author: Jordi Torres Vallverdú

Goal: idea is to replicate the analysis done by Mourifié et al (2020) in another context with another data. The idea is to seek new applications of their methodology.

Notes on the context: I am using data on Belgium region of Flandes in the 1990s. I have a panel of students observed from the beginning of high-school up to two years into higher education. Given that I don't observe future labor outcomes, I will only consider potential outcomes in terms of success probability of having completed the first year of higher education (this is a proxy for completion)


$D=\{1,0\}$, where 1 is entered stem high school track and 0 if entered non-stem high-school track

$Y=\{1,0\}$, where 1 is graduated from stem degree and 0 is graduated from non-stem degree

Z, set of instruments


In [30]:
using StatFiles, DataFrames

data_path = "/Users/jorditorresvallverdu/Library/Mobile Documents/com~apple~CloudDocs/tse/year1/papers_final_memoire/data/clean/roymodel_database.dta"

export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/set_identification/"


main = DataFrame(load(data_path))

Row,ID,hs_course1,hs_stem1,postsec_type2,postsec_stem2,c_par_beliefpar_HS_E,c_par_beliefpar_postHS,c_par_childage_homechange,c_par_notsureifB,c_par_yearsinkindergarten,c_par_repeatelementaryeduc,c_par_specialeducelem,c_par_income,c_par_home,c_par_spokenlanguage_dutch_fa,c_par_dialect_fa,c_par_spokenlanguage_dutch_mo,c_par_dialect_mo,c_par_parentschoose,c_par_childchoose,c_par_schoolchoose,c_par_otherchoose,c_par_nochoose,c_bao_advies_school_p0,c_bao_advies_progr_p0,c_bao_advies_welkprogr_p0,c_bao_belief_lkr_endpoint_p0,c_bao_special_p0,c_bao_result_dutch_p0,c_bao_result_math_p0,c_bao_result_all_p0,c_iq_male,c_iq_birth_year,c_iq_birth_month,c_iq_WB1,c_iq_NB1,c_iq_NIQ,c_iq_VIQ,c_iq_SIQ,c_iq_IQ,c_iq_opl,c_sskills_CON1,c_sskills_CON2,c_sskills_AGREE1,c_sskills_AGREE2,c_sskills_EXTRA1,c_sskills_CON3,c_sskills_AGREE3,c_sskills_EXTRA2,c_sskills_EXTRA3,c_sskills_workparents,c_sskills_income,c_sskills_IQ_discrete,z_numeric_iq
,Int32?,Float32?,Float32?,Float32?,Float32?,Int32?,Int32?,Int8?,Int8?,Int8?,Int8?,Int8?,Int32?,Int32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Int8?,Int8?,Int32?,Int32?,Float32?,Int32?,Int32?,Int32?,Int8?,Int16?,Int8?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Int8?,Float32?,Float32?,Float32?,Int8?
1,11135,6.0,1.0,2.0,0.0,6,4,missing,0,3,0,0,4,1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1,1,7,0.0,5,4,4,1,1978,8,-0.418461,0.609975,91.4869,120.331,109.205,110.171,2,5,5,4,3,4,5,2,4,3,1.0,3.0,5.0,1
2,11137,6.0,0.0,2.0,0.0,6,4,missing,0,3,0,0,5,1,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0,1,1,1,0.0,5,4,5,0,1978,10,-0.332319,0.560348,97.4658,107.186,100.036,102.72,1,5,5,5,5,5,5,5,5,5,1.0,4.0,4.0,1
3,11154,6.0,1.0,1.0,0.0,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,1,1,1,7,0.0,5,5,5,1,1978,5,0.57565,1.49024,106.991,115.378,104.474,111.563,2,5,5,5,5,5,5,5,5,5,missing,1.0,5.0,3
4,11161,6.0,1.0,1.0,0.0,6,4,missing,0,4,0,0,5,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,5,missing,0,1978,2,0.221887,1.49837,110.288,115.582,99.4507,111.189,3,5,5,5,5,5,5,5,5,5,1.0,4.0,5.0,3
5,11162,6.0,1.0,1.0,0.0,6,5,missing,0,3,0,0,4,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,5,5,0,1978,2,0.988978,0.905644,118.363,100.334,90.7207,103.559,1,5,5,5,5,5,5,5,5,4,1.0,3.0,4.0,4
6,11163,6.0,1.0,2.0,0.0,6,4,missing,0,3,0,0,5,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,6,6,missing,0,1978,6,0.959115,0.984085,120.458,118.775,126.786,125.161,3,5,5,5,5,5,5,5,5,2,1.0,4.0,6.0,5
7,11166,6.0,1.0,2.0,0.0,6,missing,missing,0,3,0,0,missing,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,5,6,missing,0,1978,11,0.158003,1.39593,95.3408,91.9245,89.5506,90.9654,3,5,5,5,5,5,5,5,5,5,missing,1.0,3.0,1
8,11171,6.0,0.0,1.0,0.0,6,3,missing,0,3,0,0,4,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1,1,1,7,0.0,6,6,missing,0,1978,5,0.83656,1.99263,110.01,125.206,121.763,123.113,2,5,5,5,5,5,5,5,5,5,1.0,3.0,6.0,3
9,11172,6.0,1.0,2.0,1.0,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,1,1,1,7,0.0,5,5,5,1,1978,10,1.0285,0.811891,108.808,96.97,99.3044,101.327,3,5,5,5,5,5,5,5,5,4,missing,1.0,4.0,3


**1. Data manipulation**

First we create a subsample of observations on gender and socio-economic status, just like they do in their exercise. I will create bins based on this: 

- Gender
- Income categories: 6 brackets
- Minority status (proxied by family background)


In [31]:
using DataFramesMeta, Statistics 

groups = groupby(main, [:c_sskills_income, :c_iq_male])
subsamples = [DataFrame(g) for g in groups]


10-element Vector{DataFrame}:
 308×54 DataFrame
 Row │ ID      hs_course1  hs_stem1  postsec_type2  postsec_stem2  c_par_belie ⋯
     │ Int32?  Float32?    Float32?  Float32?       Float32?       Int32?      ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  11166         6.0       1.0            2.0            0.0              ⋯
   2 │  11246         6.0       0.0            1.0            0.0
   3 │  11272         6.0       1.0            2.0            0.0
   4 │  11311         6.0       1.0            2.0            0.0
   5 │  11457         6.0       0.0            2.0            0.0              ⋯
   6 │  11473         6.0       0.0            2.0            0.0
   7 │  12652         6.0       0.0            2.0            0.0
   8 │  12666         4.0       0.0            2.0            0.0             
  ⋮  │   ⋮         ⋮          ⋮            ⋮              ⋮                 ⋮  ⋱
 302 │  42322         4.0       0.0            2.0       

**2. Test for monothonicity of E(Y/Z=z)**
This boils down to testing whether the probability of Y=1 is increasing in Z. Usual tests include Hu, Shi (2019), Chetverikhov (2013). These tests, however, are done under much more complicated cases as in mine: Hu et al, for example, account for a setting with Y being continuous and Z also being continuous; in my case, I just need to test over Z=1,2,3...

Given that the idea is to use set identification techniques, I will provide some evidence that this holds or not for a given subset of the data. Then I will focus mostly on the el(z) measure. 

In [32]:
size_subsample=10
size_p=3

m= zeros(Float64, size_subsample, size_p, size_p)

10×3×3 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [33]:
##Estimate probabilities 
zvals = sort(unique(main.c_iq_opl))

function cell_probs(df)
    k= length(zvals)
    p_k= zeros(Float64, k)
    n_k= zeros(Int, k)
    for z in zvals
        num = sum((df.postsec_stem2 .== 1) .& (df.c_iq_opl .== z))
        n_k[z] = sum(df.c_iq_opl .== z)
        p_k[z] = num / n_k[z]
    end

    return p_k, n_k
end


probs= [cell_probs(i) for i in subsamples]

10-element Vector{Tuple{Vector{Float64}, Vector{Int64}}}:
 ([0.2545454545454545, 0.37037037037037035, 0.45555555555555555], [110, 108, 90])
 ([0.5487804878048781, 0.6075949367088608, 0.5648148148148148], [82, 79, 108])
 ([0.20967741935483872, 0.30434782608695654, 0.358974358974359], [124, 69, 39])
 ([0.5757575757575758, 0.5263157894736842, 0.6842105263157895], [66, 38, 19])
 ([0.28, 0.38095238095238093, 0.313953488372093], [75, 126, 86])
 ([0.5964912280701754, 0.5492957746478874, 0.7090909090909091], [57, 71, 55])
 ([0.16666666666666666, 0.3333333333333333, 0.3221476510067114], [18, 69, 149])
 ([0.5714285714285714, 0.5434782608695652, 0.5504587155963303], [7, 46, 109])
 ([0.16666666666666666, 0.26666666666666666, 0.3744075829383886], [6, 30, 211])
 ([0.75, 0.4, 0.5642857142857143], [4, 15, 140])

In [34]:
#Estimate test

function pav3(pk, nk)
    p1, p2, p3 = pk
    n1, n2, n3 = nk

  #monothonicity holds
  if p1 ≤ p2 ≤ p3
    return pk
  end


  #case 1
  if p1>=p2
    
    θ_12= (p1*n1 + p2*n2)/(n1+n2)

    if p3>=θ_12
      return[θ_12, θ_12, p3]
    else 
    θ_123= (p1*n1 + p2*n2 + p3*n3)/(n1+n2+n3)

    return[θ_123,θ_123,θ_123]
    end 
  end 
  
  #case 2
  if p2>=p3

    θ_23= (p2*n2 + p3*n3)/(n2+n3)

    if p1<θ_23
      return[p1, θ_23, θ_23]
    else 
      θ_123= (p1*n1 + p2*n2 + p3*n3)/(n1+n2+n3)
     return[θ_123,θ_123,θ_123]
    end 

  end 


end


pav3 (generic function with 1 method)

In [35]:
#To check it works: 

pk = [0.40, 0.20, 0.50]
nk = [50, 60, 40]

pav3(pk, nk)


3-element Vector{Float64}:
 0.2909090909090909
 0.2909090909090909
 0.5

In [36]:
p_hat= zeros(length(subsamples), length(zvals))

for i in 1:length(subsamples)

    pk, nk = probs[i]

    p_hat[i,:]= pav3(pk,nk)
end 

p_hat

10×3 Matrix{Float64}:
 0.254545  0.37037   0.455556
 0.54878   0.582888  0.582888
 0.209677  0.304348  0.358974
 0.557692  0.557692  0.684211
 0.28      0.353774  0.353774
 0.570312  0.570312  0.709091
 0.166667  0.325688  0.325688
 0.54717   0.54717   0.550459
 0.166667  0.266667  0.374408
 0.473684  0.473684  0.564286

In [37]:
n=zeros(length(subsamples))
T=zeros(length(subsamples))
[n[i]= nrow(subsamples[i]) for i in 1:length(subsamples)]

for i in 1:length(subsamples)

    pk, nk= probs[i]
    p_hat[i,:]= pav3(pk,nk)

    T[i]= sqrt(n[i])* maximum(pk .- p_hat[i,:])
end

T

10-element Vector{Float64}:
 0.0
 0.40522880291089897
 0.0
 0.20035351497592802
 0.46043800446491184
 0.35413926923896866
 0.11744871174111007
 0.30876360795746793
 0.0
 3.4842095325169518

In [38]:
for i in 1:length(subsamples)
    println((i, unique(subsamples[i].c_sskills_income), unique(subsamples[i].c_iq_male), T[i]))
end


(1, Union{Missing, Float32}[1.0], Union{Missing, Int8}[0], 0.0)
(2, Union{Missing, Float32}[1.0], Union{Missing, Int8}[1], 0.40522880291089897)
(3, Union{Missing, Float32}[2.0], Union{Missing, Int8}[0], 0.0)
(4, Union{Missing, Float32}[2.0], Union{Missing, Int8}[1], 0.20035351497592802)
(5, Union{Missing, Float32}[3.0], Union{Missing, Int8}[0], 0.46043800446491184)
(6, Union{Missing, Float32}[3.0], Union{Missing, Int8}[1], 0.35413926923896866)
(7, Union{Missing, Float32}[4.0], Union{Missing, Int8}[0], 0.11744871174111007)
(8, Union{Missing, Float32}[4.0], Union{Missing, Int8}[1], 0.30876360795746793)
(9, Union{Missing, Float32}[5.0], Union{Missing, Int8}[0], 0.0)
(10, Union{Missing, Float32}[5.0], Union{Missing, Int8}[1], 3.4842095325169518)


In [39]:
using Distributions
using Random
Random.seed!(12345)   #seed


b_iter= 1000 # bootstrap

T_dist= zeros(length(subsamples), b_iter)

##set seed!!!

for b in 1:b_iter
  for i in 1:length(subsamples)

        dt = subsamples[i]   ##then we loop over i 
        p_iso = p_hat[i,:]   
        Z = dt.c_iq_opl  
        p_obs = p_iso[Z]   
        Y_b = rand.(Bernoulli.(p_obs))

        dt_boot = deepcopy(dt)       
        dt_boot.postsec_stem2 = Y_b   #simul replace

        pk_b, nk_b= cell_probs(dt_boot)
        p_iso_b= pav3(pk_b,nk_b)
        T_dist[i,b]= sqrt(n[i])* maximum(pk_b .- p_iso_b)
  end 
  
end

T_dist





10×1000 Matrix{Float64}:
 0.0        0.0        0.0       0.0       …  0.0       0.0       0.0
 0.636154   0.972549   0.031086  0.0          0.0       0.953155  0.537928
 0.0        0.0        0.0       0.582527     0.0       0.318857  0.337252
 0.0161575  0.0775562  0.0       0.27791      0.0       0.0       0.384549
 0.0        0.0        0.133185  0.0          0.0       0.20929   0.0
 0.0630405  0.338486   0.114956  0.0       …  0.0       0.0       0.0
 0.139917   0.0        0.0       0.0          0.666982  1.52377   0.0
 0.0        0.910412   0.962181  0.673435     3.68144   1.46545   1.88689
 0.0        0.0        0.545612  0.0          0.873124  0.0       1.30969
 0.995488   0.995488   1.3006    6.63659      2.32281   0.995488  1.65915

In [40]:
using Statistics

c_value_95= zeros(length(subsamples))
c_value_90= zeros(length(subsamples))

[c_value_95[i]= quantile(T_dist[:, i], 0.95) for i in 1:length(subsamples)]
[c_value_90[i]= quantile(T_dist[:, i], 0.90) for i in 1:length(subsamples)]



10-element Vector{Float64}:
 0.6720871825838702
 0.9748430580737397
 0.9960232920303841
 1.2697504853059818
 0.9635740940884393
 0.6942899273252582
 1.419076775323242
 1.476151328197445
 0.9988607967282772
 0.3302910284711844

In [41]:
t_result95= zeros(length(subsamples))
t_result90= zeros(length(subsamples))

for i in 1:length(subsamples)
    if (T[i]>=c_value_95[i])
        t_result95[i]=1
    else 
        t_result95[i]=0
    end

    if (T[i]>=c_value_90[i])
        t_result90[i]=1
    else 
        t_result90[i]=0
    end


end 

tests= [t_result95 t_result90]

10×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 1.0  1.0

**Using IQ as instrument**-->classic Manski Pepper type of instrument






**2. Estimating bounds on the L(z) measure**

The goal is now to understand what explains the violations of Roy selection model using the same method as in Mourifié, Henry, Meango (2020). Will keep the same example going, but, if needed, I will also include a continuous instrument: example can be numeric IQ estimated at baseline. 

The main issue with this literature is that sample analogs of $\Theta_I$ are the suprema and infima of estimated bounding functions and they have substantial finite sample bias and estimated bounds tend to be much tighter than population bounds. -->Manski/Pepper (2001).

I think I need to treat the instrument as continuous; if not it is meaningless. Because CRL seems to be devised with this property. Then I probably need to adapt the test for monothonicity too. But so far, I have something, which is fine.

Start simple; support is discrete; then we do the same exercise with a continuous instrument. 

In [42]:
##Estimate probabilities 

function cell_probs_0(df, instrument::Symbol)

    zvals = sort(unique(df[!, instrument]))  
    k = length(zvals)

    p_k = Dict{Int, Float64}()   
    n_k = Dict{Int, Int}()

    for z in zvals
        sub = df[df[!, instrument] .== z, :]

        n = nrow(sub)
        n_k[z] = n

        if n == 0
            p_k[z] = NaN
        else
            p_k[z] = sum(sub.postsec_stem2 .== 0) / n
        end
    end

    return zvals, p_k, n_k
end


probs= [cell_probs_0(i, :c_iq_opl) for i in subsamples]



10-element Vector{Tuple{Vector{Union{Missing, Int8}}, Dict{Int64, Float64}, Dict{Int64, Int64}}}:
 ([1, 2, 3], Dict(2 => 0.6296296296296297, 3 => 0.5444444444444444, 1 => 0.7454545454545455), Dict(2 => 108, 3 => 90, 1 => 110))
 ([1, 2, 3], Dict(2 => 0.3924050632911392, 3 => 0.4351851851851852, 1 => 0.45121951219512196), Dict(2 => 79, 3 => 108, 1 => 82))
 ([1, 2, 3], Dict(2 => 0.6956521739130435, 3 => 0.6410256410256411, 1 => 0.7903225806451613), Dict(2 => 69, 3 => 39, 1 => 124))
 ([1, 2, 3], Dict(2 => 0.47368421052631576, 3 => 0.3157894736842105, 1 => 0.42424242424242425), Dict(2 => 38, 3 => 19, 1 => 66))
 ([1, 2, 3], Dict(2 => 0.6190476190476191, 3 => 0.686046511627907, 1 => 0.72), Dict(2 => 126, 3 => 86, 1 => 75))
 ([1, 2, 3], Dict(2 => 0.4507042253521127, 3 => 0.2909090909090909, 1 => 0.40350877192982454), Dict(2 => 71, 3 => 55, 1 => 57))
 ([1, 2, 3], Dict(2 => 0.6666666666666666, 3 => 0.6778523489932886, 1 => 0.8333333333333334), Dict(2 => 69, 3 => 149, 1 => 18))
 ([1, 2, 3], Dict(

In [43]:
function compute_el(df, instrument::Symbol)

    zvals, p_k, n_k = cell_probs_0(df, instrument)

    el = zeros(length(zvals))

    running_min = Inf

    for (i, z) in enumerate(zvals)
        running_min = min(running_min, p_k[z])
        el[i] = p_k[z] - running_min
    end

    return zvals, p_k, n_k, el
end


zvals, p_k, n_k, el = compute_el(subsamples[10], :c_iq_opl)


(Union{Missing, Int8}[1, 2, 3], Dict(2 => 0.6, 3 => 0.4357142857142857, 1 => 0.25), Dict(2 => 15, 3 => 140, 1 => 4), [0.0, 0.35, 0.18571428571428572])

In [44]:
#Generate moment inequalities

function m_ineq(theta, p_k, n_k, zvals_s)
    n = length(zvals_s)
    moment = zeros(n,n)
    se_moment = zeros(n,n)

    for ii in 1:n
        z_i = zvals_s[ii]

        for jj in 1:ii
            z_j = zvals_s[jj]

            if n_k[z_i] == 0 || n_k[z_j] == 0
                moment[ii,jj] = 0.0
                se_moment[ii,jj] = Inf
                continue
            end

            moment[ii,jj] = (p_k[z_i] - p_k[z_j]) - theta[ii] ##key point
            var_i = p_k[z_i]*(1 - p_k[z_i]) / n_k[z_i]
            var_j = p_k[z_j]*(1 - p_k[z_j]) / n_k[z_j]

            se_moment[ii,jj] = sqrt(var_i + var_j)
        end
    end

    return moment, se_moment
end



m_ineq (generic function with 1 method)

In [45]:
#put the moment inequalities in a vector

function vectorize(moment::AbstractMatrix)
 n = size(moment, 1)

 K = n * (n+ 1) ÷ 2
 moment_vector= zeros(K)
    k=1
        for j in 1:n
            for y in 1:j
                    moment_vector[k]=moment[j,y]
                    k +=1
            end 
            
        end
     
  return moment_vector

end 

vectorize (generic function with 1 method)

In [46]:
##Now generate T value of the moment for a given value of the grid. 

function T_test(df, instrument, theta)

    zvals, p_k, n_k, el= compute_el(df, instrument)

        
    moment, se_moment=m_ineq(theta, p_k, n_k, zvals)

    moment_v= vectorize(moment)
    moment_se= vectorize(se_moment)

    T_value= maximum(moment_v./max.(moment_se, 1e-8))
         
    return T_value

end 

T_test (generic function with 1 method)

In [47]:

function bootstrap(df, moment_not,se_moment_not,theta_not, B, instrument::Symbol, zvals_original)

    T_b= zeros(B)
    for b in 1:B

        df_b = nothing
        while true
            n = nrow(df)
            df_try = df[rand(1:n, n), :]
            zvals_b = sort(unique(df_try[!, instrument]))
            if zvals_b == zvals_original
                df_b = df_try
                break
            end
        end

        zvals, p_k_b, n_k_b, el_b= compute_el(df_b, instrument)
        
        moment, se_moment=m_ineq(theta_not, p_k_b, n_k_b, zvals)

        moment_v_b= vectorize(moment)

        Z_vec_b= @. (moment_v_b - moment_not) /se_moment_not
        T_b[b]=maximum(Z_vec_b)

    end 
    return T_b

end 

bootstrap (generic function with 1 method)

In [48]:
#let's take only the case where it is violated, the main condition...



zvals, p_k, n_k, el = compute_el(subsamples[10], :c_iq_opl)

theta_not=el 
moment_not, se_moment_not= m_ineq(theta_not, p_k, n_k, zvals)

se_not_vec= vectorize(se_moment_not)
moment_not_vec=vectorize(moment_not)


zvals_original= sort(unique(subsamples[10][!, :c_iq_opl]))



3-element Vector{Union{Missing, Int8}}:
 1
 2
 3

In [49]:
T_boot= bootstrap(subsamples[10], moment_not_vec,se_not_vec,theta_not, 1000, :c_iq_opl, zvals_original)


K_alpha= quantile(T_boot, 0.95)



1.9786881975072133

In [50]:
##Apply the code for each subsample and for different instruments. 


function apply_bootsubsamples(instrument::Symbol, B)

    size_s= length(subsamples)
    T_boot_s=zeros(size_s, B)
    
    for s in 1:size_s
        
        zvals, p_k, n_k, el = compute_el(subsamples[s], instrument)

        theta_not=el #should I change this?
        moment_not, se_moment_not= m_ineq(theta_not, p_k, n_k, zvals)

        se_not_vec= vectorize(se_moment_not)
        moment_not_vec=vectorize(moment_not)


        zvals_original= sort(unique(subsamples[s][!, instrument]))

        T_boot_s[s, :]= bootstrap(subsamples[s], moment_not_vec,se_not_vec,theta_not, B, instrument, zvals_original)

        

    end 

    return T_boot_s

end

apply_bootsubsamples (generic function with 1 method)

In [51]:
T_boot_s= apply_bootsubsamples(:c_iq_opl, 1000)


iz= length(subsamples)

alpha_by_subsample= zeros(iz)

[alpha_by_subsample[i]= quantile(T_boot_s[i,:], 0.95) for i in 1:iz]





10-element Vector{Float64}:
 2.1601960170312497
 2.0738836126693445
 2.0564537092948756
 2.1579000526168595
 2.0832948842084993
 2.1478974921286333
 2.106672011149204
 2.2103463347416534
 2.3278194005813493
 1.8118805392327646

In [52]:
##Finally set up the grid, compute T and then compare with critical values simulated via bootstrap. 

#set a grid of values for el(z) do I need to loop over these or jus tby computing the upper bound is enough... ? is it really needed more?

In [53]:
function T_test(df, instrument, theta)

    zvals, p_k, n_k, el= compute_el(df, instrument)

        
    moment, se_moment=m_ineq(theta, p_k, n_k, zvals)

    moment_v= vectorize(moment)
    moment_se= vectorize(se_moment)

    T_value= maximum(moment_v./max.(moment_se, 1e-8))
         
    return T_value

end 

T_test (generic function with 1 method)

In [54]:
T_test(subsamples[10], :c_iq_opl, theta_not)

theta_not

3-element Vector{Float64}:
 0.0
 0.35
 0.18571428571428572

In [55]:
#compute te

function T_simul(subsamples, θ_grid, instrument::Symbol)

    iz = length(subsamples)              
    grid_size = length(θ_grid)           

    T_grid = zeros(iz, grid_size)

    for z in 1:iz
        df_s = subsamples[z]
        for theta_idx in 1:grid_size
            θ = θ_grid[theta_idx]
            T_grid[z, theta_idx] = T_test(df_s, instrument, θ)
        end
    end

    return T_grid
end


T_simul (generic function with 1 method)

In [56]:

grid1 = [0.0]                     # dimension 1
grid2 = collect(0.0:0.005:0.35)   # dimension 2
grid3 = collect(0.0:0.005:0.185)   # dimension 3


θ_grid = [ [x,y,z] for (x,y,z) in Iterators.product(grid1, grid2, grid3) ]

T_grid= T_simul(subsamples, θ_grid, :c_iq_opl)






10×2698 Matrix{Float64}:
 0.0       0.0       0.0       0.0       …  0.0        0.0        0.0
 0.587967  0.587967  0.587967  0.587967     0.0        0.0        0.0
 0.0       0.0       0.0       0.0          0.0        0.0        0.0
 0.488074  0.438716  0.389357  0.339999     0.0        0.0        0.0
 1.0128    1.0128    1.0128    1.0128       0.0        0.0        0.0
 0.53751   0.480565  0.42362   0.366675  …  0.0        0.0        0.0
 0.1634    0.1634    0.1634    0.1634       0.0        0.0        0.0
 0.139094  0.114212  0.108642  0.108642     0.0        0.0        0.0
 0.0       0.0       0.0       0.0          0.0        0.0        0.0
 1.39582   1.37588   1.35594   1.336        0.0398805  0.0199403  0.00323903

In [57]:
function accepted_theta_per_subsample(T_grid, alpha_by_subsample, θ_grid)

    ns = size(T_grid, 1)      
    ng = size(T_grid, 2)       

    accepted = Vector{Vector{Vector{Float64}}}(undef, ns)

    for s in 1:ns
        k_s = alpha_by_subsample[s]

        θ_accepted = Vector{Vector{Float64}}()

        for g in 1:ng
            if T_grid[s, g] <= k_s
                push!(θ_accepted, θ_grid[g])
            end
        end

        accepted[s] = θ_accepted
    end

    return accepted
end


accepted_theta_per_subsample (generic function with 1 method)

In [58]:
T_grid = T_simul(subsamples, θ_grid, :c_iq_opl)

accepted = accepted_theta_per_subsample(T_grid, alpha_by_subsample, θ_grid)


10-element Vector{Vector{Vector{Float64}}}:
 [[0.0, 0.0, 0.0], [0.0, 0.005, 0.0], [0.0, 0.01, 0.0], [0.0, 0.015, 0.0], [0.0, 0.02, 0.0], [0.0, 0.025, 0.0], [0.0, 0.03, 0.0], [0.0, 0.035, 0.0], [0.0, 0.04, 0.0], [0.0, 0.045, 0.0]  …  [0.0, 0.305, 0.185], [0.0, 0.31, 0.185], [0.0, 0.315, 0.185], [0.0, 0.32, 0.185], [0.0, 0.325, 0.185], [0.0, 0.33, 0.185], [0.0, 0.335, 0.185], [0.0, 0.34, 0.185], [0.0, 0.345, 0.185], [0.0, 0.35, 0.185]]
 [[0.0, 0.0, 0.0], [0.0, 0.005, 0.0], [0.0, 0.01, 0.0], [0.0, 0.015, 0.0], [0.0, 0.02, 0.0], [0.0, 0.025, 0.0], [0.0, 0.03, 0.0], [0.0, 0.035, 0.0], [0.0, 0.04, 0.0], [0.0, 0.045, 0.0]  …  [0.0, 0.305, 0.185], [0.0, 0.31, 0.185], [0.0, 0.315, 0.185], [0.0, 0.32, 0.185], [0.0, 0.325, 0.185], [0.0, 0.33, 0.185], [0.0, 0.335, 0.185], [0.0, 0.34, 0.185], [0.0, 0.345, 0.185], [0.0, 0.35, 0.185]]
 [[0.0, 0.0, 0.0], [0.0, 0.005, 0.0], [0.0, 0.01, 0.0], [0.0, 0.015, 0.0], [0.0, 0.02, 0.0], [0.0, 0.025, 0.0], [0.0, 0.03, 0.0], [0.0, 0.035, 0.0], [0.0, 0.04, 0.0], [